In [11]:
import os
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [12]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
import networkx as nx
from langchain.chains import GraphQAChain
from langchain_core.documents import Document
from langchain_community.graphs.networkx_graph import NetworkxEntityGraph

llm = AzureChatOpenAI(
    azure_deployment="gpt-35-turbo",  
    api_version="2024-08-01-preview", 
    temperature=0.7,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

text = """
Marie Curie, born in 1867, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.
She was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.
Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
She was, in 1906, the first woman to become a professor at the University of Paris. 
"""
print(len(text))

575


In [4]:
documents = [Document(page_content=text)]
llm_transformer = LLMGraphTransformer(llm=llm)
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [5]:
graph = NetworkxEntityGraph()

# Add nodes to the graph
for node in graph_documents[0].nodes:
    graph.add_node(node.id)

# Add edges to the graph
for edge in graph_documents[0].relationships:
    graph._graph.add_edge(
            edge.source.id,
            edge.target.id,
            relation=edge.type,
        )

In [6]:
chain = GraphQAChain.from_llm(
    llm=llm, 
    graph=graph, 
    verbose=True
)

In [7]:
question = """Who is Marie Curie?"""
chain.invoke(question)

/home/nikunj/Downloads/IRE/Project/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new GraphQAChain chain...
Entities Extracted:
Marie Curie
Full Context:
Marie Curie BORN_IN 1867
Marie Curie NATIONALITY Polish
Marie Curie NATURALIZED_AS French
Marie Curie OCCUPATION Physicist
Marie Curie OCCUPATION Chemist
Marie Curie RESEARCH_ON Radioactivity
Marie Curie WON Nobel Prize
Marie Curie WON Nobel Prize In Two Scientific Fields
Marie Curie MARRIED_TO Married Couple
Marie Curie BECAME_A_PROFESSOR_AT University Of Paris

> Finished chain.


'Marie Curie is a Polish-born physicist and chemist who became a French citizen and conducted groundbreaking research on radioactivity. She won the Nobel Prize in two scientific fields and was married to a fellow scientist. She also became a professor at the University of Paris.'